In [8]:
import json
import pandas as pd
from pprint import pprint

import utils.run_t5_mlm_flax

TypeError: dataclass_transform() got an unexpected keyword argument 'field_specifiers'

In [2]:

with open('./datasets/raw/personachat_self_original.json','r') as f:
    a = json.loads(f.read())

In [3]:
def persona_samples(file, split='train'):
    for dialogue in a[split]:
        persona = " ".join(dialogue ['personality'])
        history = dialogue['utterances'][-1]['history']
        # dio_samples = ["<persona> " + persona + "<history> " + history[i] for i in range(0,len(history),2)]
        curr_history = []
        for i in range(1,len(history),2):
            curr_history.append(history[i-1])
            yield "<persona> " + persona + "< history> " + " ".join(curr_history) + " <answer> " + history[i]
            curr_history.append(history[i])
    


In [4]:
with open("./datasets/processed/personaAI/train.json", 'w') as f:
    for sample in persona_samples(a):
        f.write(json.dumps({"text":sample} ))
        f.write("\n")



In [5]:
with open("./datasets/processed/personaAI/valid.json", 'w') as f:
    for sample in persona_samples(a,split='valid'):
        f.write(json.dumps({"text":sample} ))
        f.write("\n")


In [6]:
from datasets import load_dataset, Features, Value
# context_feat = Features({'text': Value(dtype='strcontext_feat = Features({'text': Value(dtype='string', id=None)})ing', id=None)})
personaAI = load_dataset('json', data_files={'train':'./datasets/processed/personaAI/train.json',
                                        'validation':"./datasets/processed/personaAI/valid.json"}
                        )

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 82.88it/s]
Generating train split: 113560 examples [00:00, 383090.45 examples/s]
Generating validation split: 6801 examples [00:00, 333023.51 examples/s]


In [8]:
personaAI['train'][0]

{'text': "<persona> i like to remodel homes . i like to go hunting . i like to shoot a bow . my favorite holiday is halloween .< history> hi , how are you doing ? i'm getting ready to do some cheetah chasing to stay in shape . <answer> you must be very fast . hunting is one of my favorite hobbies ."}

In [41]:
print(personaAI['train'][0]['text'])
tokenizer.decode(tokenizer(personaAI['train'][0]['text']).input_ids)

<persona> i like to remodel homes . i like to go hunting . i like to shoot a bow . my favorite holiday is halloween .< history> hi , how are you doing ? i'm getting ready to do some cheetah chasing to stay in shape . <answer> you must be very fast . hunting is one of my favorite hobbies .


"<unk>persona> i like to remodel homes. i like to go hunting. i like to shoot a bow. my favorite holiday is halloween.<unk> history> hi, how are you doing? i'm getting ready to do some cheetah chasing to stay in shape. <unk>answer> you must be very fast. hunting is one of my favorite hobbies.</s>"